In [64]:
import requests
import os
import json
from dotenv import load_dotenv
import pathlib
import pandas as pd

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'

# name the file we want
env_file_name = 'lrhf97_twit_cred.env'

# designate the path of the file
curr_file = pathlib.Path.cwd()

# join them
env_path = curr_file.joinpath(env_file_name)

# load the file
load_dotenv(env_path)

bearer_token = os.getenv("BEARER_TOKEN")


def create_url():
    # Replace with user ID below - RidgeSupply =3413455348
    user_id = 3413455348
    
    return "https://api.twitter.com/2/users/{}/followers?max_results=10".format(user_id)

def get_params():
    return {"user.fields": "username"}

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FollowersLookupPython"
    return r


def connect_to_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url = create_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params)
    # print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response


ridge_followers= main()

200


In [65]:
df=pd.DataFrame. from_dict(ridge_followers['data'])
df

,id,name,username
0,14387509,frankgiraffe,frankgiraffe
1,33535739,Jordan Spoon,jspoon13
2,1418283365577461764,f3morganton,f3morganton
3,1416820910712135682,DSRPT cycling,CyclingDsrpt
4,1268533564196478987,Steven Boyd,boyd_steven_s
5,52964262,Daniel De Nobrega,dandng
6,385446082,Timothy Anderson,tanderson999
7,877695112884633601,velokicks,velokicks
8,20304670,Damian Howe,_munimula_
9,1162847621225910272,Wookie,wookieapproved


In [66]:
usernames = ','.join(list(df['username']))
usernames

'frankgiraffe,jspoon13,f3morganton,CyclingDsrpt,boyd_steven_s,dandng,tanderson999,velokicks,_munimula_,wookieapproved'

In [68]:

# Get the bios of the followers

def create_user_url():
    # Replace with username
    # user_id = 15903747str(df['username'][0])','str(df['username'][1]))
    usernames = ','.join(list(df['username']))

# https://api.twitter.com/2/users/by?usernames=TwitterDev,Twitter

    return "https://api.twitter.com/2/users/by?usernames={}".format(usernames)
    # return "https://api.twitter.com/2/users/by?usernames=frankgiraffe,jspoon13"

def get_user_params():
    return {"user.fields":"id",
            "user.fields":"description"}


def connect_to_user_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main_user():
    url = create_user_url()
    params = get_user_params()
    json_user_response = connect_to_user_endpoint(url, params)
    return json_user_response

follower_user_data = main_user()


200


In [69]:
df_user=pd.DataFrame. from_dict(follower_user_data['data'])
df_user


,description,id,username,name
0,Living at the intersection of geekery and the ...,14387509,frankgiraffe,frankgiraffe
1,Slinging paints on cars for @maconisetupshop -...,33535739,jspoon13,Jordan Spoon
2,"Fitness, Fellowship, & Faith coming to Morgant...",1418283365577461764,f3morganton,f3morganton
3,,1416820910712135682,CyclingDsrpt,DSRPT cycling
4,"@UChicago polisci grad student - IR, formal th...",1268533564196478987,boyd_steven_s,Steven Boyd
5,Caraqueño,52964262,dandng,Daniel De Nobrega
6,"Rider of bikes, and drinker of beer. I like to...",385446082,tanderson999,Timothy Anderson
7,Have a weakness for shoes? So do we.,877695112884633601,velokicks,velokicks
8,What can i say..................,20304670,_munimula_,Damian Howe
9,🚴 ☕️ 🎮 🥘 📷 🪴 🥖 #adhd,1162847621225910272,wookieapproved,Wookie


In [84]:

def create_liked_url():
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    tweet_fields = "tweet.fields=text,author_id"
    user_fields = "user.fields=username"
    # Be sure to replace your-user-id with your own user ID or one of an authenticating user
    # You can find a user ID by using the user lookup endpoint
    id = 3413455348
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    liked_url = "https://api.twitter.com/2/users/{}/liked_tweets".format(id)
    return liked_url, tweet_fields, user_fields

def connect_to_liked_endpoint(liked_url, tweet_fields):
    response = requests.request(
        "GET", liked_url, auth=bearer_oauth, params=tweet_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def main():
    liked_url, tweet_fields, user_fields = create_liked_url()
    json_response = connect_to_liked_endpoint(liked_url, tweet_fields)
    return json_response

liked_texts =main()
liked_texts

200


{'data': [{'author_id': '1527870560',
   'id': '1105094316194705408',
   'text': 'New bike day! Video up on my YouTube channel. Link in bio. @ninerbikes @familybikeshop @ridgesupply @handupgloves @starlightbikes @trainright #ninerbikes #jet9rdo #rlt9rdo #rkt9rdo @… https://t.co/ZPq3fBLOMc'},
  {'author_id': '2691291337',
   'id': '1105131276384780288',
   'text': 'We’re honestly just crazy excited to be heading to @croatanbuckfifty this week. We have a few days of actual work to power through first but then it’s ocean, gravel, NASCAR track, lots of @ridgesupply, #rodeobeachouse vol II, a huge gathering of Rodeo folks, and best of all … https://t.co/7rN0zbPVDb'},
  {'author_id': '247961183',
   'id': '1106596392200478720',
   'text': '#stokebox pickup is hoppin at the croatanbuckfifty right now !! #peeps from all over rolling in for the #bestbikegameever @ridgesupply @Rodeo_Labs wahoofitnessofficial #shakethedust @… https://t.co/gJFmbWtVMw'},
  {'author_id': '247961183',
   'id': '11066

In [86]:

def create_url():
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    user_fields = "user.fields=created_at,description"
    # You can replace the ID given with the Tweet ID you wish to lookup Retweeting users for
    # You can find an ID by using the Tweet lookup endpoint
    id = "1437987598790582277"
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets/{}/retweeted_by".format(id)
    return url, user_fields


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RetweetedByPython"
    return r


def connect_to_endpoint(url, user_fields):
    response = requests.request("GET", url, auth=bearer_oauth, params=user_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url, user_fields = create_url()
    json_response = connect_to_endpoint(url, user_fields)
    return json_response

retweets = main()
retweets



200


{'data': [{'username': 'bambamman12',
   'description': 'Ex-military. Huge Bama Fan. Patriot and lover of freedom and democracy. Believe in honor and integrity in politics. Oppose corruption and dishonesty.',
   'created_at': '2010-10-05T21:10:03.000Z',
   'name': 'Gerald D Tinnon',
   'id': '199027092'},
  {'username': 'richwinterhoff',
   'description': "Love sports,music,and anything that's FUN !!",
   'created_at': '2015-02-20T13:16:42.000Z',
   'name': 'NYCsportsnut',
   'id': '3046752167'},
  {'username': 'pennywise56',
   'description': "I'm Sorry I Hurt Your Feelings When I Called You Stupid I Really Thought You Already Knew",
   'created_at': '2021-09-03T02:14:36.000Z',
   'name': 'pennywisethecat',
   'id': '1433614208206245891'},
  {'username': 'DarrinHall15',
   'description': 'Trying to make a better world for my kids\n#Facts #Science #Ethics\nWear a Mask & Get Vaccinated! #BlueWave2022',
   'created_at': '2016-08-19T21:14:01.000Z',
   'name': 'Darrin Hall',
   'id': '7667